<a href="https://colab.research.google.com/github/Srinidhi231/activity/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# list files (adjust the path to where you added the shortcut)
!ls -lh "/content/drive/MyDrive/CSV"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 1.3G
-rw------- 1 root root 5.8M Aug 26 08:12 'Copy of sample_submission.csv'
-rw------- 1 root root  25M Aug 26 08:11 'Copy of test_identity.csv'
-rw------- 1 root root 585M Aug 26 08:11 'Copy of test_transaction.csv'
-rw------- 1 root root  26M Aug 26 08:11 'Copy of train_identity.csv'
-rw------- 1 root root 652M Aug 26 08:11 'Copy of train_transaction.csv'


In [3]:
!pip install -q pandas numpy scikit-learn lightgbm category_encoders imbalanced-learn shap matplotlib seaborn pyarrow
# Optional for big data: dask[complete] or vaex


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.1 MB/s eta 0:00:00


In [7]:
import pandas as pd
train_tr = pd.read_csv('/content/drive/MyDrive/CSV/Copy of train_transaction.csv', nrows=200000)
train_id = pd.read_csv('/content/drive/MyDrive/CSV/Copy of train_identity.csv', nrows=200000)


In [9]:
import numpy as np
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum()/1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
    end_mem = df.memory_usage().sum()/1024**2
    print(f"Memory: {start_mem:.2f} MB -> {end_mem:.2f} MB")
    return df


In [10]:
df = train_tr.copy()
df.info()
df.head()
df.describe()
# missing values
miss = df.isna().mean().sort_values(ascending=False)
print(miss.head(30))
# target distribution (train_transaction should have 'isFraud')
train_tr['isFraud'].value_counts(normalize=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 601.2+ MB
D7       0.936120
dist2    0.922405
D13      0.897960
D14      0.888805
D12      0.886315
D6       0.868525
D9       0.823420
D8       0.823420
M7       0.814550
M8       0.814550
M9       0.814550
V139     0.753210
V153     0.753210
V150     0.753210
V154     0.753210
V152     0.753210
V151     0.753210
V146     0.753210
V145     0.753210
V144     0.753210
V143     0.753210
V142     0.753210
V149     0.753210
V148     0.753210
V147     0.753210
V158     0.753210
V138     0.753210
V165     0.753210
V166     0.753210
V163     0.753210
dtype: float64


,proportion
isFraud,
0,0.96988
1,0.03012


In [11]:
!pip install -q pandas numpy scikit-learn lightgbm imbalanced-learn category_encoders pyarrow matplotlib seaborn


In [12]:
from google.colab import drive
drive.mount('/content/drive')

# point to your "CSV" folder
DATA_DIR = "/content/drive/MyDrive/CSV"

TRAIN_TR = f"{DATA_DIR}/Copy of train_transaction.csv"
TRAIN_ID = f"{DATA_DIR}/Copy of train_identity.csv"
TEST_TR  = f"{DATA_DIR}/Copy of test_transaction.csv"
TEST_ID  = f"{DATA_DIR}/Copy of test_identity.csv"
SAMPLE_SUB = f"{DATA_DIR}/Copy of sample_submission.csv"

!ls -lh "$DATA_DIR"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 1.3G
-rw------- 1 root root 5.8M Aug 26 08:12 'Copy of sample_submission.csv'
-rw------- 1 root root  25M Aug 26 08:11 'Copy of test_identity.csv'
-rw------- 1 root root 585M Aug 26 08:11 'Copy of test_transaction.csv'
-rw------- 1 root root  26M Aug 26 08:11 'Copy of train_identity.csv'
-rw------- 1 root root 652M Aug 26 08:11 'Copy of train_transaction.csv'


In [13]:
import pandas as pd, numpy as np, gc, joblib
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type.kind in ("i","u","f"):
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type)[:3] == 'int' or str(col_type)[:3] == 'uin':
                for t in (np.int8, np.int16, np.int32, np.int64):
                    if c_min >= np.iinfo(t).min and c_max <= np.iinfo(t).max:
                        df[col] = df[col].astype(t); break
            else:
                for t in (np.float16, np.float32):
                    try:
                        df[col] = df[col].astype(t); break
                    except: pass
    end_mem = df.memory_usage().sum() / 1024**2
    # print(f"Mem: {start_mem:.1f} MB -> {end_mem:.1f} MB")
    return df

def safe_read_csv(path, **kwargs):
    print("Loading:", path)
    return pd.read_csv(path, low_memory=False, **kwargs)


In [14]:
train_tr = safe_read_csv(TRAIN_TR)
train_id = safe_read_csv(TRAIN_ID)
test_tr  = safe_read_csv(TEST_TR)
test_id  = safe_read_csv(TEST_ID)

print("shapes:", train_tr.shape, train_id.shape, test_tr.shape, test_id.shape)

train = train_tr.merge(train_id, on="TransactionID", how="left")
test  = test_tr.merge(test_id, on="TransactionID", how="left")
del train_tr, train_id, test_tr, test_id
gc.collect()

train = reduce_mem_usage(train)
test  = reduce_mem_usage(test)

print("Merged shapes:", train.shape, test.shape)
print("Target distribution:\n", train['isFraud'].value_counts(normalize=True))


Loading: /content/drive/MyDrive/CSV/Copy of train_transaction.csv
Loading: /content/drive/MyDrive/CSV/Copy of train_identity.csv
Loading: /content/drive/MyDrive/CSV/Copy of test_transaction.csv
Loading: /content/drive/MyDrive/CSV/Copy of test_identity.csv
shapes: (590540, 394) (144233, 41) (506691, 393) (141907, 41)


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes

Merged shapes: (590540, 434) (506691, 433)
Target distribution:
 isFraud
0    0.96501
1    0.03499
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [15]:
# 4.1 create datetime (if TransactionDT exists in original dataset)
START_DATE = pd.to_datetime("2017-12-01")  # conventional anchor used in many kernels
def add_time_features(df):
    if 'TransactionDT' in df.columns:
        df['datetime'] = START_DATE + pd.to_timedelta(df['TransactionDT'], unit='s')
    else:
        # try to find a timestamp column
        found = False
        for c in df.columns:
            if 'date' in c.lower() or 'time' in c.lower():
                try:
                    df['datetime'] = pd.to_datetime(df[c])
                    found = True
                    break
                except:
                    continue
        if not found:
            df['datetime'] = pd.NaT
    # add time parts if available
    df['DT_hour'] = df['datetime'].dt.hour.fillna(-1).astype('int16')
    df['DT_dayofweek'] = df['datetime'].dt.dayofweek.fillna(-1).astype('int16')
    df['TransactionAmt_log1p'] = np.log1p(df['TransactionAmt'].fillna(0)).astype('float32')
    return df

train = add_time_features(train)
test  = add_time_features(test)

# 4.2 select categorical columns we will use
base_cats = ['card1','card2','addr1','P_emaildomain','R_emaildomain']
base_cats = [c for c in base_cats if c in train.columns]

# 4.3 frequency maps (fit on train+test)
freq_maps = {}
for col in base_cats:
    all_vals = pd.concat([train[col].fillna("##na##"), test[col].fillna("##na##")], axis=0)
    vc = all_vals.value_counts(dropna=False)
    freq_maps[col] = vc.to_dict()
    # attach freq to train/test
    train[col + "_freq"] = train[col].fillna("##na##").map(freq_maps[col]).fillna(0).astype('float32')
    test[col + "_freq"]  = test[col].fillna("##na##").map(freq_maps[col]).fillna(0).astype('float32')

# 4.4 category -> int maps (simple mapping dicts)
cat_maps = {}
for col in base_cats:
    all_vals = pd.concat([train[col].fillna("##na##"), test[col].fillna("##na##")], axis=0).astype(str)
    uniques = pd.Series(all_vals.unique())
    m = {v: i+1 for i,v in enumerate(uniques)}   # reserve 0 for missing/unseen
    cat_maps[col] = m
    train[col + "_code"] = train[col].fillna("##na##").astype(str).map(m).fillna(0).astype('int32')
    test[col + "_code"]  = test[col].fillna("##na##").astype(str).map(m).fillna(0).astype('int32')

# 4.5 define final feature list (intersection with test)
candidate_features = [
    'TransactionAmt_log1p','DT_hour','DT_dayofweek'
] + [c + '_freq' for c in base_cats] + [c + '_code' for c in base_cats]

features = [f for f in candidate_features if f in train.columns and f in test.columns]
print("Using features:", features)


/tmp/ipython-input-2726068978.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['datetime'] = START_DATE + pd.to_timedelta(df['TransactionDT'], unit='s')
/tmp/ipython-input-2726068978.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['DT_hour'] = df['datetime'].dt.hour.fillna(-1).astype('int16')
/tmp/ipython-input-2726068978.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

Using features: ['TransactionAmt_log1p', 'DT_hour', 'DT_dayofweek', 'card1_freq', 'card2_freq', 'addr1_freq', 'P_emaildomain_freq', 'R_emaildomain_freq', 'card1_code', 'card2_code', 'addr1_code', 'P_emaildomain_code', 'R_emaildomain_code']


/tmp/ipython-input-2726068978.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[col + "_code"] = train[col].fillna("##na##").astype(str).map(m).fillna(0).astype('int32')
/tmp/ipython-input-2726068978.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[col + "_code"]  = test[col].fillna("##na##").astype(str).map(m).fillna(0).astype('int32')


In [17]:
import os

# Define the directory where you want to save models
MODEL_DIR = f"{DATA_DIR}/models"

# Create the directory if it doesn't exist
os.makedirs(MODEL_DIR, exist_ok=True)

# Now save your objects
joblib.dump(freq_maps, f"{MODEL_DIR}/freq_maps.joblib")
joblib.dump(cat_maps,  f"{MODEL_DIR}/cat_maps.joblib")

print("✅ Saved freq_maps and cat_maps to models/")


✅ Saved freq_maps and cat_maps to models/


In [20]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

X = train[features].fillna(-1)
y = train['isFraud'].astype('int8')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(train), dtype='float32')
models = []

params = {
    "objective":"binary",
    "metric":"auc",
    "learning_rate":0.05,
    "num_leaves":64,
    "feature_fraction":0.8,
    "bagging_fraction":0.8,
    "bagging_freq":1,
    "min_data_in_leaf":100,
    "verbosity":-1,
    "seed":42,
    "n_jobs":-1,
    "is_unbalance":True
}

for fold, (tr_idx, va_idx) in enumerate(skf.split(X,y),1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dval = lgb.Dataset(X_va, label=y_va)

model = lgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    valid_sets=[dval],
    callbacks=[
        early_stopping(150),
        log_evaluation(200)  # print logs every 200 iterations
    ]
)
oof[va_idx] = model.predict(X_va, num_iteration=model.best_iteration)
models.append(model)
joblib.dump(model, f"{DATA_DIR}/models/lgb_fold{fold}.joblib")
print(f"Saved model lgb_fold{fold}.joblib")

print("CV AUC:", roc_auc_score(y, oof))
joblib.dump(models, f"{DATA_DIR}/models/lgb_models_all.joblib")
print("Saved all models.")


Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.883281
[400]	valid_0's auc: 0.900591
[600]	valid_0's auc: 0.909229
[800]	valid_0's auc: 0.914415
[1000]	valid_0's auc: 0.917951
[1200]	valid_0's auc: 0.920431
[1400]	valid_0's auc: 0.922337
[1600]	valid_0's auc: 0.923704
[1800]	valid_0's auc: 0.924776
[2000]	valid_0's auc: 0.925399
[2200]	valid_0's auc: 0.926122
[2400]	valid_0's auc: 0.926453
[2600]	valid_0's auc: 0.927138
[2800]	valid_0's auc: 0.92754
[3000]	valid_0's auc: 0.927943
[3200]	valid_0's auc: 0.928296
[3400]	valid_0's auc: 0.928674
[3600]	valid_0's auc: 0.928868
[3800]	valid_0's auc: 0.929145
Early stopping, best iteration is:
[3812]	valid_0's auc: 0.929243
Saved model lgb_fold5.joblib
CV AUC: 0.5171813588969114
Saved all models.


In [29]:
# load saved maps + models
freq_maps = joblib.load(f"{DATA_DIR}/models/freq_maps.joblib")
cat_maps  = joblib.load(f"{DATA_DIR}/models/cat_maps.joblib")
models    = joblib.load(f"{DATA_DIR}/models/lgb_models_all.joblib")

def preprocess_user_df(df):
    # ensure TransactionID
    if 'TransactionID' not in df.columns:
        df.insert(0, 'TransactionID', range(1, len(df)+1))

    # time features
    if 'TransactionDT' in df.columns:
        df['datetime'] = START_DATE + pd.to_timedelta(df['TransactionDT'], unit='s')
    else:
        parsed = False
        for c in df.columns:
            if 'date' in c.lower() or 'time' in c.lower():
                try:
                    df['datetime'] = pd.to_datetime(df[c])
                    parsed = True
                    break
                except:
                    continue
        if not parsed:
            df['datetime'] = pd.NaT

    df['DT_hour'] = df['datetime'].dt.hour.fillna(-1).astype('int16')
    df['DT_dayofweek'] = df['datetime'].dt.dayofweek.fillna(-1).astype('int16')
    df['TransactionAmt_log1p'] = np.log1p(df['TransactionAmt'].fillna(0)).astype('float32')

    # safe freq maps
    for col in freq_maps:
        col_freq = col + '_freq'
        if col in df.columns:
            series = df[col]
        else:
            series = pd.Series(["##na##"] * len(df))
        df[col_freq] = (
            series.fillna("##na##")
            .map(freq_maps[col])
            .fillna(0)
            .astype('float32')
        )

    # safe cat maps
    for col in cat_maps:
        col_code = col + '_code'
        if col in df.columns:
            series = df[col].astype(str)
        else:
            series = pd.Series(["##na##"] * len(df))
        df[col_code] = (
            series.fillna("##na##")
            .map(cat_maps[col])
            .fillna(0)
            .astype('int32')
        )

    # ensure feature columns exist
    for f in features:
        if f not in df.columns:
            df[f] = -1  # fallback
    return df


def predict_user_file(user_csv_path, proba_col='isFraud_proba', threshold=0.5):
    user = pd.read_csv(user_csv_path, low_memory=False)
    user = preprocess_user_df(user)
    X_user = user[features].fillna(-1)

    # average fold predictions
    preds = np.mean([m.predict(X_user, num_iteration=m.best_iteration) for m in models], axis=0)
    user[proba_col] = preds
    user['isFraud_flag'] = (preds >= threshold).astype(int)

    out_csv = str(Path(user_csv_path).parent / "user_results.csv")
    user.to_csv(out_csv, index=False)
    print("Saved results to:", out_csv)
    return user


# Example usage (after you drop user_transactions.csv into DATA_DIR)
user_file = f"{DATA_DIR}/user_transactions.csv"
if Path(user_file).exists():
    user_df = predict_user_file(user_file, threshold=0.5)
    display(user_df.head(10))
else:
    print("Put your user CSV as 'user_transactions.csv' in the CSV folder and re-run this cell.")


Saved results to: /content/drive/MyDrive/CSV/user_results.csv


,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card4,ProductCD,datetime,DT_hour,...,addr1_freq,P_emaildomain_freq,R_emaildomain_freq,card1_code,card2_code,addr1_code,P_emaildomain_code,R_emaildomain_code,isFraud_proba,isFraud_flag
0,1,86400,50.00,12345,150,150,visa,W,2017-12-01,0,...,131315.0,163648.0,824070.0,4542,0,10,1,1,0.000884,0
1,2,172800,200.75,23456,200,150,mastercard,C,2017-12-02,0,...,131315.0,163648.0,824070.0,0,0,10,1,1,0.000619,0
2,3,259200,500.00,34567,250,150,discover,R,2017-12-03,0,...,131315.0,163648.0,824070.0,0,0,10,1,1,0.001113,0


from matplotlib import pyplot as plt
_df_0['TransactionID'].plot(kind='hist', bins=20, title='TransactionID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['TransactionDT'].plot(kind='hist', bins=20, title='TransactionDT')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['TransactionAmt'].plot(kind='hist', bins=20, title='TransactionAmt')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['ProductCD'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='TransactionID', y='ProductCD', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['card4'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='TransactionDT', y='card4', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['ProductCD'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='TransactionDT', y='ProductCD', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [22]:
def plot_basic_reports(user_df, proba_col='isFraud_proba', show_top_n=15):
    # histogram of predicted probabilities
    plt.figure(figsize=(8,4))
    plt.hist(user_df[proba_col], bins=50)
    plt.title("Histogram of predicted fraud probability")
    plt.xlabel("Predicted probability"); plt.ylabel("Count")
    plt.show()

    # pie chart flagged vs not flagged
    counts = user_df['isFraud_flag'].value_counts().sort_index()
    plt.figure(figsize=(5,5))
    plt.pie(counts, labels=['Not flagged','Flagged'], autopct='%1.1f%%', startangle=140)
    plt.title("Flagged vs Not flagged (thresholded)")
    plt.show()

    # time-series of flagged counts by day (if datetime exists)
    if 'datetime' in user_df.columns:
        tmp = user_df.dropna(subset=['datetime']).copy()
        if not tmp.empty:
            ts = tmp.set_index('datetime').resample('D')['isFraud_flag'].sum()
            plt.figure(figsize=(10,3))
            ts.plot(marker='o')
            plt.title("Daily count of flagged transactions")
            plt.ylabel("Flagged count"); plt.show()

    # top risky transactions table
    top = user_df.sort_values(by=proba_col, ascending=False).head(show_top_n)
    display(top[['TransactionID','TransactionAmt','datetime',proba_col,'isFraud_flag'] + [c for c in base_cats if c in top.columns]])

    # top cards by average risk
    if 'card1' in user_df.columns:
        agg = user_df.groupby('card1')[proba_col].mean().sort_values(ascending=False).head(10)
        plt.figure(figsize=(6,4))
        agg.plot(kind='barh')
        plt.title("Top 10 card1 by average fraud probability")
        plt.xlabel("avg probability"); plt.gca().invert_yaxis(); plt.show()

# If user_df exists:
if 'user_df' in globals():
    plot_basic_reports(user_df)
else:
    print("No user_df found; run predict_user_file() first.")


No user_df found; run predict_user_file() first.
